<a href="https://colab.research.google.com/github/Alexey1998-ml/tasks/blob/master/recomendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!unzip drive/MyDrive/nlp/archive.zip -d drive/MyDrive/nlp/

Archive:  drive/MyDrive/nlp/archive.zip
replace drive/MyDrive/nlp/reddit_data.csv? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [ ]:
!git clone https://github.com/linanqiu/reddit-dataset.git

fatal: destination path 'reddit-dataset' already exists and is not an empty directory.


In [ ]:
!pip install hnswlib

In [ ]:
import os
import pandas as pd
import datetime
import math
from gensim.models import FastText
import numpy as np
import hnswlib
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
def load_big_text_df(path_to_df_dir):
  """
  Делаем один большой DataFrame
  """
  header=['text','id','subreddit','meta','time','author','ups','downs',
          'authorlinkkarma','authorkarma','authorisgold']
  df = None
  for name in os.listdir(path_to_df_dir):
    print(name, end='\r')
    if name.split('.')[-1] == 'csv':
      current_df = pd.read_csv(os.path.join(path_to_df_dir, name), header=0, index_col=0).iloc[1:, -11:]
      current_df.columns = header
      if df is None:
        df = current_df
      else:
        df = pd.concat([df, current_df], axis=0)
  # return df[['text', 'subreddit', 'author']].dropna()
  print('Loaded')
  return df.dropna()

In [ ]:
text_author_topic_df = load_big_text_df('reddit-dataset')

Loaded


In [ ]:
user_topic_df = pd.read_csv('drive/MyDrive/nlp/reddit_data.csv')

Так как в датасете с github есть username и subreddict, которых нету в датасете с kaggle и наоборот, обрежим датасеты таким образом, что бы все username и subreddict пересекались

In [ ]:
def make_cross_sets(df_1, df_2, column_1, column_2):
  """
  Получаем значения в колонках, которые есть в первом и втором df
  """
  set_1 = set(df_1[column_1].values)
  set_2 = set(df_2[column_2].values)
  return set_1 & set_2

In [ ]:
cross_username = make_cross_sets(df_1=text_author_topic_df, df_2=user_topic_df, column_1='author', column_2='username')
cross_topic = make_cross_sets(df_1=text_author_topic_df, df_2=user_topic_df, column_1='subreddit', column_2='subreddit')

In [ ]:
text_author_topic_df['match_row'] = text_author_topic_df[['author', 'subreddit']].apply(lambda x: True if (x[0] in cross_username and x[1] in cross_topic) else False, axis=1)
user_topic_df['match_row'] = user_topic_df[['username', 'subreddit']].apply(lambda x: True if (x[0] in cross_username and x[1] in cross_topic) else False, axis=1)

In [ ]:
# обрезаем датасеты
text_author_topic_df = text_author_topic_df[text_author_topic_df['match_row'] == True]
user_topic_df = user_topic_df[user_topic_df['match_row'] == True]

In [ ]:
# для наглядности переведем время
text_author_topic_df['time'] = text_author_topic_df['time'].apply(lambda x: datetime.datetime.fromtimestamp(x))
user_topic_df['time'] = user_topic_df['utc'].apply(lambda x: datetime.datetime.fromtimestamp(x))

Сразу отложим тестовые данные. Считаем, что корпус с github это тренировочные данные, а данные с kaggle - тестовые. Для того, что бы не предсказывать прошлое, уберем из последнего датасета топики, у которых время обращения раньше, чем самое позднее время, которое есть в тренировочном наборе для соответствующего пользователя 

In [ ]:
def make_test_dict(text_author_topic_df, user_topic_df, cross_username):
  """
  Генерируем словарь {username: [topic1, topic2 .. ]}  
  """
  max_time_dict = dict(text_author_topic_df.groupby(['author', 'time'])['time'].max().index)
  test_dict = {}
  for username in cross_username:
    print(username, end='\r')
    user_topic_df_slice = user_topic_df[user_topic_df['username'] == username] 
    test_topics = user_topic_df_slice[user_topic_df_slice['time'] > max_time_dict[username]]['subreddit'].values
    test_dict[username] = list(set(test_topics))
  return test_dict

In [ ]:
cross_username = make_cross_sets(df_1=text_author_topic_df, df_2=user_topic_df, column_1='author', column_2='username')

In [ ]:
test_dict = make_test_dict(text_author_topic_df=text_author_topic_df, user_topic_df=user_topic_df, cross_username=cross_username)

In [ ]:
# удаляем username, у которых нету топиков, удовлетворяющих нашим условиям в 
# датасете с kaggle
drop_username = []
for username in test_dict:
  if len(test_dict[username]) == 0:
    drop_username.append(username)
    print(username)
cross_username -= set(drop_username)
text_author_topic_df = text_author_topic_df[text_author_topic_df['author'].apply(lambda x: x in cross_username)]

zuperpailon


# Немного предобработаем текст

In [ ]:
lemmatizer = WordNetLemmatizer()

In [ ]:
special_char = ',./_=\!?'
def delete_special_char(text):
  """
  Удаляем специальные символы
  """
  for char in special_char:
    text = text.replace(char, '')
  return text

In [ ]:
set_stopwords = set(stopwords.words('english'))  # чтоб искать за O(1)
def process_text(text):
  """
  Удаляем специальные символы, токинизируем и лемматизируем текст
  :return [word1, word2, ..]
  """
  text = delete_special_char(text)
  text = word_tokenize(text)
  return [lemmatizer.lemmatize(word) for word in text if not word in set_stopwords]

In [ ]:
text_author_topic_df['words'] = text_author_topic_df['text'].apply(process_text)

In [ ]:
# Некоторые пользователи пишут бессмысленные посты, состоящие только из stopwords
text_author_topic_df = text_author_topic_df[text_author_topic_df['words'].apply(lambda x: x != [])]
cross_username = make_cross_sets(df_1=text_author_topic_df, df_2=user_topic_df, column_1='author', column_2='username')
cross_topic = make_cross_sets(df_1=text_author_topic_df, df_2=user_topic_df, column_1='subreddit', column_2='subreddit')

# Учим FastText

In [ ]:
embedding_dim = 128
fasttext_model = FastText(size=embedding_dim, window=3, seed=42, min_count=5)

In [ ]:
fasttext_model.build_vocab(text_author_topic_df['words'])

In [ ]:
fasttext_model.train(sentences=text_author_topic_df['words'], total_examples=len(text_author_topic_df), epochs=10) 

# Модель интересов пользователя и модель, описывающая топики


In [ ]:
class TextItemModel:
  """
  В качестве item может выступать либо пользователь либо топик
  Каждый пользователь и топик характеризуется плотным вектором, 
  который получается путем вычисления среднего значения эмбединга текстов, 
  которые относятся к конкретному item
  Для пользователя - среднее значение эмбединга всех его текстов
  Для топика - среднее значение эмбеддинга всех тектов на данный топик
  """

  def __init__(self, fasttext_model, text_author_topic_df, group_column, words_column='words'):
    self.model = fasttext_model
    self.df = text_author_topic_df
    self.group_column = group_column
    self.words_column = words_column
  
  def __embedding_from_sentence(self, words):
    """
    Среднее значение эмбединга текста
    :return np.array(size=embedding_dim)
    """
    return np.sum(np.array(list(map(self.model.__getitem__, words))), axis=0)

  def get_item_vector(self, item):
    """
    Среднее значение эмбединга для всех текстов
    :return np.array(size=embedding_dim)
    """
    return self.df[self.df[self.group_column] == item][self.words_column].apply(self.__embedding_from_sentence).mean(axis=0)

In [ ]:
text_user_model = TextItemModel(fasttext_model=fasttext_model, text_author_topic_df=text_author_topic_df, group_column='author')

In [ ]:
text_topic_model = TextItemModel(fasttext_model=fasttext_model, text_author_topic_df=text_author_topic_df, group_column='subreddit')

# Рекомендательная система

In [ ]:
class RecomendationSystem:
  """
  Класс, позволяющий ранжировать топики для конкретных пользователей
  При инициализации получаем вектора эмбедингов для всех топиков и строим 
  на них граф, в котором можно будет эффективно искать ближайших соседей O(log(n))
  В камечестве меры близости считается косинусная близость - 1 - cos(v1, v2)
  """

  def __init__(self, text_topic_model, text_user_model):
    topic_coords = []
    topic_dict = {}
    for idx, topic in enumerate(cross_topic):
      topic_coords.append(text_topic_model.get_item_vector(topic))
      topic_dict[idx] = topic
    graph = hnswlib.Index(space='cosine', dim=embedding_dim)
    graph.init_index(len(cross_topic), M = 16, ef_construction = 200, random_seed = 100)
    graph.add_items(np.array(topic_coords), range(len(topic_coords)))
    self.topic_dict = topic_dict
    self.graph = graph
    self.text_user_model = text_user_model

  def get_topics(self, username, k=10):
    """
    k - число наиболее близких топиков, которые необходимо вывести
    :return np.array([topic_1, topic_2, ..]), np.array([1 - cos(user, topic_1), 1 - cos(user, topic_2), ..])
    """
    user_embedding = self.text_user_model.get_item_vector(username)
    neighbors, similarity = self.graph.knn_query(user_embedding, k=10)
    topics = np.array(list(map(self.topic_dict.__getitem__, neighbors[0])))
    return topics, similarity

In [ ]:
recomend_sys = RecomendationSystem(text_topic_model=text_topic_model, text_user_model=text_user_model)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
recomend_sys.get_topics(username=list(cross_username)[0])


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


(array(['thewalkingdead', 'politics', 'funny', 'skyrim', 'startrek',
        'guns', 'food', 'explainlikeimfive', 'motorcycles', 'comicbooks'],
       dtype='<U17'),
 array([[0.38961184, 0.403373  , 0.40477908, 0.40536213, 0.40542394,
         0.40674466, 0.41098273, 0.4191075 , 0.4209404 , 0.4217881 ]],
       dtype=float32))

# Валидация
Допустим, что у нас есть возможность показать пользователю 10 различных топиков, тогда в качестве метрики будем использовать число топиков выведенных топиков (из тестового датасета), на которые в дальнейшем пользователь написал пост

In [ ]:
predict_dict = {}
for user in cross_username:
  topics, _ = recomend_sys.get_topics(user)
  predict_dict[user] = topics

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).


In [ ]:
cross_topics_validation = []
for user in predict_dict:
  cross_topics_validation.append(len(set(predict_dict[user]) & set(test_dict[user])))

In [ ]:
np.mean(cross_topics_validation)

2.2454819277108435

Какое бы значение имела данная метрика, если бы мы случайно угадывали топики?

In [ ]:
def combinations(n, k):
  """
  Число сочетаний
  """
  return math.factorial(n) / (math.factorial(k) * math.factorial(n - k))

In [47]:
def p(n, m, k, good_topics):
  """
  Вероятность того, что число правильно выведенных топиков равно good_topics
  n - всего топиков, m - число топиков, на которые пользователь написал пост
  k = 10 - число выводимых топиков
  """
  p = 1
  for i in range(good_topics):
    p *= (m - i) / (n - i)
    if i > k:
      return p
  for i in range(0, k - good_topics):
    p *= (n - m - i)/(n - i - good_topics)
  if k < good_topics:
    good_topics = k
  return combinations(k, good_topics) * p 

In [ ]:
def expectation_good_topics(n, m, k):
  """
  Математическое ожидание числа правильно показанных топиков
  """
  expectation = 0
  for i in range(1, m + 1):
    expectation += i * p(n, m, k, i)
  return expectation

In [ ]:
# проверим все ли хорошо с формулами
all_topics = list(range(40))
good_topics = list(range(5))
match = []
for i in range(10000):
  topics = np.random.choice(all_topics, size = 10, replace=False)
  match.append(len(set(topics) & set(good_topics)))
print(f'Действительное значение: {np.mean(match)}')
print(f'Теоретическое: {expectation_good_topics(40, 5, 10)}')

Действительное значение: 1.2555
Теоретическое: 1.2499999999999998


In [51]:
for user in cross_username:
  topics = np.random.choice(list(cross_topic), size=10, replace=False)
  predict_dict[user] = topics

In [52]:
cross_topics_validation_random = []
for user in predict_dict:
  cross_topics_validation_random.append(len(set(predict_dict[user]) & set(test_dict[user])))

In [53]:
np.mean(cross_topics_validation_random)  # брут форс

1.7846385542168675

In [49]:
total_topics = len(cross_topic)
mean_good_topics = []
k = 10
for user in cross_username:
  m = len(test_dict[user])
  mean_good_topics.append(expectation_good_topics(total_topics, m, k))

In [50]:
np.mean(mean_good_topics)  # по формуле

1.7987106085153497

In [43]:
np.mean(cross_topics_validation)  # модель

2.2454819277108435

Модель предсказывает лучше случайного выбора, но не на много